In [19]:
import numpy as np
import cv2
import easyocr

In [20]:
# Initialize EasyOCR reader once
reader = easyocr.Reader(['en'])
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [21]:
def select_meeting_type():
    while True:
        print("Select meeting type:")
        print("1. Google Meet")
        print("2. Zoom")
        choice = input("Enter your choice (1 or 2): ")
        if choice in ['1', '2']:
            return choice
        print("Invalid choice. Please try again.")

def process_contour_region(frame, x, y, w, h, prev_text=""):
    roi = frame[y:y+h, x:x+w]
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray_roi,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    
    for (fx, fy, fw, fh) in faces:
        cv2.rectangle(roi, (fx, fy), (fx+fw, fy+fh), (255, 255, 0), 2)
    
    results = reader.readtext(roi)
    text = ' '.join([result[1] for result in results])
    
    if text.strip() and text.strip() != prev_text.strip():
        print(f"Text Detected: {text.strip()}")
        if len(faces) > 0:
            print(f"Number of faces detected: {len(faces)}")
        return text.strip(), len(faces) > 0
    return prev_text, len(faces) > 0

def process_both_rois(frame, contour_coords=None):
    # Fixed ROI coordinates
    x1, y1 = 1055, 1
    x2, y2 = 1274, 126
    
    # Process fixed ROI
    fixed_roi = frame[y1:y2, x1:x2]
    gray_fixed_roi = cv2.cvtColor(fixed_roi, cv2.COLOR_BGR2GRAY)
    fixed_faces = face_cascade.detectMultiScale(
        gray_fixed_roi,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )
    
    # Only perform OCR if faces detected in fixed ROI
    fixed_results = []
    if len(fixed_faces) > 0:
        fixed_results = reader.readtext(fixed_roi)
        if fixed_results:
            print("\nFixed ROI OCR Results (Face Detected):")
            for result in fixed_results:
                print(f"Text: {result[1]}, Confidence: {result[2]:.2f}")
    
    # Draw faces in fixed ROI
    for (fx, fy, fw, fh) in fixed_faces:
        cv2.rectangle(fixed_roi, (fx, fy), (fx+fw, fy+fh), (255, 255, 0), 2)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Process contour ROI if available
    contour_faces = []
    contour_results = []
    if contour_coords:
        x, y, w, h = contour_coords
        contour_roi = frame[y:y+h, x:x+w]
        gray_contour_roi = cv2.cvtColor(contour_roi, cv2.COLOR_BGR2GRAY)
        contour_faces = face_cascade.detectMultiScale(
            gray_contour_roi,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
        
        # Only perform OCR if faces detected in contour ROI
        if len(contour_faces) > 0:
            contour_results = reader.readtext(contour_roi)
            if contour_results:
                print("\nContour ROI OCR Results (Face Detected):")
                for result in contour_results:
                    print(f"Text: {result[1]}, Confidence: {result[2]:.2f}")
        
        # Draw faces in contour ROI
        for (fx, fy, fw, fh) in contour_faces:
            cv2.rectangle(contour_roi, (fx, fy), (fx+fw, fy+fh), (255, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    return fixed_results, contour_results, len(contour_faces) > 0

def main():
    meeting_type = select_meeting_type()
    cap = cv2.VideoCapture('samples/green.mp4')
    show_fixed_roi = True
    
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 1920, 1080)
    cv2.moveWindow('frame', 0, 0)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        if meeting_type == '2':  # Zoom
            lower_green = np.array([40,50,50])
            upper_green = np.array([80,255,255])
            mask = cv2.inRange(hsv, lower_green, upper_green)
            cnts = cv2.findContours(mask.copy(),
                                  cv2.RETR_EXTERNAL,
                                  cv2.CHAIN_APPROX_SIMPLE)[-2]
            
            contour_coords = None
            if len(cnts) > 0:
                area = max(cnts, key=cv2.contourArea)
                contour_coords = cv2.boundingRect(area)
            
            fixed_results, contour_results, face_in_contour = process_both_rois(frame, contour_coords)
            
            # Display OCR results
            if contour_coords and contour_results:
                x, y, _, _ = contour_coords
                text = ' '.join([result[1] for result in contour_results])
                cv2.putText(frame, text[:50], (x, y-10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            
            if fixed_results and show_fixed_roi:
                text = ' '.join([result[1] for result in fixed_results])
                cv2.putText(frame, text[:50], (1055, 0), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
            
            # Update ROI visibility
            show_fixed_roi = not face_in_contour

        try:
            cv2.imshow('frame', frame)
            key = cv2.waitKey(5) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame + 30)
        except Exception as e:
            print(f"Error displaying frame: {e}")
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Select meeting type:
1. Google Meet
2. Zoom



Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Brown, Confidence: 1.00

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Brown, Confidence: 1.00

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.65

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.65

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.65

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.65

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.70

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.61

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.93

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.93

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Browh, Confidence: 0.82

Contour ROI OCR Results (Face Detected):
Text: Cr Andrew Brown, Confidence: 0.55

Contour ROI OCR